In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Dataframes
command: disjoint speaker split

In [3]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/speech_commands/'
command_df = pd.read_pickle(os.path.join(dataframe_dir,'Command_Dataframe.pkl'))

In [4]:
import honk_sv.train as hk
from honk_sv import model as mod

# SI Train

### Dataset split

In [5]:
#TODO: correct splits
all_spks = command_df.spk.unique()
all_sents = command_df.sent.unique()
uttrs_counts = command_df.spk.value_counts()
si_spks = list(command_df.spk.unique())

In [7]:
import random 

si_df = command_df

# command dataset is disjoint along speakers.
si_spk_train = si_df[si_df.set == 'train']
si_spk_val = si_df[si_df.set == 'val']
si_spk_test = si_df[si_df.set == 'test']
print("[spk] train:{}, val:{}, test:{}".format(len(si_spk_train), len(si_spk_val), len(si_spk_test)))

# spliting along the sents
random.shuffle(all_sents)
train_sent = [sent for sent in all_sents if sent not in ['yes', 'no', 'up', 'down']]
# train_sent = all_sents[:20]
val_sent = ['yes', 'no']
test_sent = ['up', 'down']
si_sent_train = si_df[si_df.sent.isin(train_sent)]
si_sent_val = si_df[si_df.sent.isin(val_sent)]
si_sent_test = si_df[si_df.sent.isin(test_sent)]
print("[sent] train:{}, val:{}, test:{}".format(len(si_sent_train), len(si_sent_val), len(si_sent_test)))

# random sampling
si_random_train = si_df.sample(frac=0.8)
si_random_test = si_df.drop(index=si_random_train.index)
si_random_val = si_random_test.sample(frac=0.5)
si_random_test = si_random_test.drop(index=si_random_val.index) 
print("[random] train:{}, val:{}, test:{}".format(len(si_random_train), len(si_random_val), len(si_random_test)))

[spk] train:51088, val:6798, test:6835
[sent] train:55235, val:4752, test:4734
[random] train:51777, val:6472, test:6472


choose 2300 as the total number of utterances 

In [37]:
subject_spks = list(command_df.spk.value_counts().index[:100])
subject_df = command_df[command_df.spk.isin(subject_spks)]

command_small_df = subject_df.sample(n=520)
si_small_train = command_small_df.sample(frac=0.8)
si_small_test = command_small_df.drop(index=si_small_train.index)
si_small_val = si_small_test.sample(frac=0.5)
si_small_test = si_small_test.drop(index=si_small_val.index) 
print("[small] train:{}, val:{}, test:{}".format(len(si_small_train), len(si_small_val), len(si_small_test)))

[small] train:416, val:52, test:52


In [45]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_small_train, 'val':si_small_val, 'test':si_small_test}
manifest_dir = "manifests/commands/"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'small_{}_{}_manifest.csv'.format("command", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.sent, row.file)
#             label = row.label
            label = subject_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

manifests/commands/small_command_train_manifest.csv was written
manifests/commands/small_command_val_manifest.csv was written
manifests/commands/small_command_test_manifest.csv was written


In [72]:
all_sents

array(['right', 'zero', 'bed', 'sheila', 'up', 'left', 'six', 'five',
       'seven', 'tree', 'two', 'marvin', 'eight', 'one', 'bird', 'stop',
       'cat', 'wow', 'yes', 'dog', 'down', 'three', 'off', 'no', 'house',
       'four', 'on', 'go', 'happy', 'nine'], dtype=object)

In [79]:
right_spks = list(command_df.spk.value_counts().index[:100])
right_df = command_df[(command_df.spk.isin(right_spks)) & (command_df.sent == 'right')]

command_word_df = right_df
si_word_train = command_word_df.sample(frac=0.8)
si_word_test = command_word_df.drop(index=si_word_train.index)
si_word_val = si_word_test.sample(frac=0.5)
si_word_test = si_word_test.drop(index=si_word_val.index) 
print("[right] train:{}, val:{}, test:{}".format(len(si_word_train), len(si_word_val), len(si_word_test)))

[right] train:415, val:52, test:52


In [81]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_word_train, 'val':si_word_val, 'test':si_word_test}
manifest_dir = "manifests/commands/"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'word_{}_{}_manifest.csv'.format("command", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.sent, row.file)
#             label = row.label
            label = yes_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

manifests/commands/word_command_train_manifest.csv was written
manifests/commands/word_command_val_manifest.csv was written
manifests/commands/word_command_test_manifest.csv was written


### Global Configuration

In [94]:
dataset = "command"
model = "cnn-small"
global_config = dict(dataset=dataset, n_labels=100,no_cuda=False, gpu_no=0, 
                     seed=0, use_nesterov=False,
                     cache_size=32768, num_workers=8)

train_config = dict(n_epochs=150, batch_size=16,
                    lr=[0.001], schedule=[np.inf], dev_every=1,
                    momentum=0.9, weight_decay=0.00001,
                    print_step=50)

model_config = dict(model=model)

In [95]:
builder = hk.ConfigBuilder(
                mod.find_config(model_config['model']),
                mod.SpeechDataset.default_config(dataset),
                global_config, train_config, model_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
si_config['model_class'] = mod.find_model(model)
hk.set_seed(si_config)

***

### Train Universal Model

In [70]:
manifest_dir = "manifests/commands/"
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)]=os.path.join(manifest_dir,'small_{}_{}_manifest.csv'.format("command", tag))

In [41]:
si_model = si_config['model_class'](si_config)
si_config['input_file'] = ""
si_config['output_file'] = "models/si_command_model.pt"

In [42]:
pp.pprint(si_config)

ChainMap({   'batch_size': 16,
             'bkg_noise_folder': '/home/muncok/DL/dataset/SV_sets/speech_commands/_background_noise_',
             'cache_size': 32768,
             'conv1_pool': [1, 1],
             'conv1_size': [101, 8],
             'conv1_stride': [1, 1],
             'data_folder': '/home/muncok/DL/dataset/SV_sets',
             'dataset': 'command',
             'dev_every': 1,
             'dnn1_size': 128,
             'dnn2_size': 128,
             'dropout_prob': 0.5,
             'gpu_no': 0,
             'height': 101,
             'input_file': '',
             'input_length': 16000,
             'lr': [0.001],
             'model': 'cnn-small',
             'model_class': <class 'honk_sv.model.SpeechModel'>,
             'momentum': 0.9,
             'n_dct_filters': 40,
             'n_epochs': 150,
             'n_feature_maps1': 16,
             'n_labels': 100,
             'n_mels': 40,
             'no_cuda': False,
             'noise_prob': 0.8,
 

In [43]:
hk.train(si_config, model=si_model)

train step #49 accuracy: 0.0, loss: 4.605052947998047
train step #99 accuracy: 0.0, loss: 4.606278419494629
epoch #0, final dev accuracy: 0.016666666666666666
saving best model...
train step #149 accuracy: 0.0, loss: 4.601314067840576
train step #199 accuracy: 0.0, loss: 4.606049537658691
epoch #1, final dev accuracy: 0.016666666666666666
train step #249 accuracy: 0.0, loss: 4.601868152618408
train step #299 accuracy: 0.0, loss: 4.600224494934082
epoch #2, final dev accuracy: 0.016666666666666666
train step #349 accuracy: 0.0625, loss: 4.594691753387451
train step #399 accuracy: 0.0, loss: 4.606830596923828
train step #449 accuracy: 0.0625, loss: 4.592522621154785
epoch #3, final dev accuracy: 0.029166666666666667
saving best model...
train step #499 accuracy: 0.0, loss: 4.615309238433838
train step #549 accuracy: 0.0, loss: 4.635168075561523
epoch #4, final dev accuracy: 0.0125
train step #599 accuracy: 0.0, loss: 4.5805230140686035
train step #649 accuracy: 0.0, loss: 4.6149740219116

train step #5049 accuracy: 0.3125, loss: 3.391425609588623
epoch #43, final dev accuracy: 0.5291666666666667
saving best model...
train step #5099 accuracy: 0.3125, loss: 2.990612268447876
train step #5149 accuracy: 0.375, loss: 2.200573205947876
epoch #44, final dev accuracy: 0.5597222222222222
saving best model...
train step #5199 accuracy: 0.5625, loss: 1.441232681274414
train step #5249 accuracy: 0.5, loss: 2.0542097091674805
epoch #45, final dev accuracy: 0.5013888888888889
train step #5299 accuracy: 0.3125, loss: 2.2325470447540283
train step #5349 accuracy: 0.25, loss: 2.9520087242126465
train step #5399 accuracy: 0.25, loss: 2.3474462032318115
epoch #46, final dev accuracy: 0.5083333333333333
train step #5449 accuracy: 0.625, loss: 2.158762216567993
train step #5499 accuracy: 0.3125, loss: 2.9788548946380615
epoch #47, final dev accuracy: 0.48194444444444445
train step #5549 accuracy: 0.3125, loss: 2.345276355743408
train step #5599 accuracy: 0.1875, loss: 2.622929096221924
epo

train step #10099 accuracy: 0.3125, loss: 1.8900296688079834
epoch #87, final dev accuracy: 0.6583333333333333
train step #10149 accuracy: 0.3125, loss: 2.584308385848999
train step #10199 accuracy: 0.625, loss: 1.6132384538650513
epoch #88, final dev accuracy: 0.7097222222222223
saving best model...
train step #10249 accuracy: 0.625, loss: 1.8436932563781738
train step #10299 accuracy: 0.375, loss: 2.3118581771850586
train step #10349 accuracy: 0.5, loss: 1.5856993198394775
epoch #89, final dev accuracy: 0.6527777777777778
train step #10399 accuracy: 0.4375, loss: 2.2583932876586914
train step #10449 accuracy: 0.375, loss: 2.347142219543457
epoch #90, final dev accuracy: 0.6833333333333333
train step #10499 accuracy: 0.5, loss: 1.68375563621521
train step #10549 accuracy: 0.4375, loss: 2.168363571166992
epoch #91, final dev accuracy: 0.6944444444444444
train step #10599 accuracy: 0.625, loss: 1.9571943283081055
train step #10649 accuracy: 0.5625, loss: 1.4160258769989014
epoch #92, fi

epoch #130, final dev accuracy: 0.701388888888889
train step #15099 accuracy: 0.5625, loss: 2.119581460952759
train step #15149 accuracy: 0.3125, loss: 2.159284830093384
epoch #131, final dev accuracy: 0.726388888888889
train step #15199 accuracy: 0.5, loss: 1.5144405364990234
train step #15249 accuracy: 0.625, loss: 1.058769702911377
epoch #132, final dev accuracy: 0.6930555555555556
train step #15299 accuracy: 0.5, loss: 1.3229598999023438
train step #15349 accuracy: 0.625, loss: 1.5696569681167603
train step #15399 accuracy: 0.5625, loss: 1.8182528018951416
epoch #133, final dev accuracy: 0.7180555555555556
train step #15449 accuracy: 0.6875, loss: 1.5886812210083008
train step #15499 accuracy: 0.5, loss: 1.597289800643921
epoch #134, final dev accuracy: 0.6916666666666667
train step #15549 accuracy: 0.5625, loss: 1.4685146808624268
train step #15599 accuracy: 0.375, loss: 1.92616605758667
epoch #135, final dev accuracy: 0.7097222222222223
train step #15649 accuracy: 0.3125, loss: 3

In [71]:
si_config['n_epochs'] = 0
si_config['input_file'] = "models/si_command_model.pt"
hk.train(si_config, model=si_model)

final test accuracy: 0.7307692307692307


***

### Train Word Model

In [98]:
manifest_dir = "manifests/commands/"
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)]=os.path.join(manifest_dir,'word_{}_{}_manifest.csv'.format("command", tag))

In [90]:
word_model = si_config['model_class'](si_config)

In [100]:
si_config['input_file'] = ""
si_config['output_file'] = "models/si_command_word_model.pt"

In [101]:
pp.pprint(si_config)

ChainMap({   'batch_size': 16,
             'bkg_noise_folder': '/home/muncok/DL/dataset/SV_sets/speech_commands/_background_noise_',
             'cache_size': 32768,
             'conv1_pool': [1, 1],
             'conv1_size': [101, 8],
             'conv1_stride': [1, 1],
             'data_folder': '/home/muncok/DL/dataset/SV_sets',
             'dataset': 'command',
             'dev_every': 1,
             'dnn1_size': 128,
             'dnn2_size': 128,
             'dropout_prob': 0.5,
             'gpu_no': 0,
             'height': 101,
             'input_file': '',
             'input_length': 16000,
             'lr': [0.001],
             'model': 'cnn-small',
             'model_class': <class 'honk_sv.model.SpeechModel'>,
             'momentum': 0.9,
             'n_dct_filters': 40,
             'n_epochs': 150,
             'n_feature_maps1': 16,
             'n_labels': 100,
             'n_mels': 40,
             'no_cuda': False,
             'noise_prob': 0.8,
 

In [102]:
hk.train(si_config, model=word_model)

epoch #0, final dev accuracy: 0.015625
saving best model...
train step #49 accuracy: 0.0, loss: 4.622599124908447
epoch #1, final dev accuracy: 0.015625
epoch #2, final dev accuracy: 0.015625
train step #99 accuracy: 0.0, loss: 4.5344557762146
epoch #3, final dev accuracy: 0.015625
epoch #4, final dev accuracy: 0.015625
train step #149 accuracy: 0.0, loss: 4.596512317657471
epoch #5, final dev accuracy: 0.015625
epoch #6, final dev accuracy: 0.015625
train step #199 accuracy: 0.0, loss: 4.628245830535889
epoch #7, final dev accuracy: 0.015625
epoch #8, final dev accuracy: 0.015625
train step #249 accuracy: 0.0625, loss: 4.568501949310303
epoch #9, final dev accuracy: 0.015625
epoch #10, final dev accuracy: 0.015625
train step #299 accuracy: 0.125, loss: 4.4036030769348145
epoch #11, final dev accuracy: 0.015625
epoch #12, final dev accuracy: 0.015625
train step #349 accuracy: 0.0, loss: 4.6794610023498535
epoch #13, final dev accuracy: 0.015625
epoch #14, final dev accuracy: 0.015625
t

epoch #115, final dev accuracy: 0.546875
epoch #116, final dev accuracy: 0.515625
train step #2949 accuracy: 0.5, loss: 1.6825355291366577
epoch #117, final dev accuracy: 0.53125
epoch #118, final dev accuracy: 0.453125
train step #2999 accuracy: 0.6875, loss: 1.0731024742126465
epoch #119, final dev accuracy: 0.625
saving best model...
epoch #120, final dev accuracy: 0.53125
train step #3049 accuracy: 0.5625, loss: 1.3333040475845337
epoch #121, final dev accuracy: 0.578125
epoch #122, final dev accuracy: 0.578125
train step #3099 accuracy: 0.4375, loss: 1.957601547241211
epoch #123, final dev accuracy: 0.625
epoch #124, final dev accuracy: 0.484375
train step #3149 accuracy: 0.5, loss: 2.5244040489196777
epoch #125, final dev accuracy: 0.703125
saving best model...
epoch #126, final dev accuracy: 0.65625
train step #3199 accuracy: 0.5, loss: 2.012476682662964
epoch #127, final dev accuracy: 0.578125
epoch #128, final dev accuracy: 0.609375
train step #3249 accuracy: 0.75, loss: 1.157

In [69]:
si_config['n_epochs'] = 0
si_config['input_file'] = "models/si_command_word_model.pt"
hk.train(si_config, model=word_model)

final test accuracy: 0.6730769230769231


In [45]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_small_train, 'val':si_small_val, 'test':si_small_test}
manifest_dir = "manifests/commands/"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'small_{}_{}_manifest.csv'.format("command", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.sent, row.file)
#             label = row.label
            label = subject_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

manifests/commands/small_command_train_manifest.csv was written
manifests/commands/small_command_val_manifest.csv was written
manifests/commands/small_command_test_manifest.csv was written


In [61]:
si_config['n_labels']

4